In [2]:
import librosa
import os
import numpy as np

In [3]:
y, sr = librosa.load("music1.wav")

# 跟踪歌曲的节奏点
tempo, beat_frames = librosa.beat.beat_track(y=y, sr=sr)
# 对提取的节奏序列进行差分
beat_frames = librosa.feature.delta(beat_frames)

print(beat_frames)

[28.73333333 28.73333333 28.73333333 28.73333333 28.73333333 28.81666667
 28.93333333 29.         29.         28.93333333 28.81666667 28.73333333
 28.68333333 28.66666667 28.61666667 28.55       28.55       28.61666667
 28.66666667 28.68333333 28.73333333 28.81666667 28.93333333 29.
 29.         28.93333333 28.81666667 28.73333333 28.68333333 28.6
 28.56666667 28.58333333 28.65       28.76666667 28.78333333 28.76666667
 28.78333333 28.76666667 28.71666667 28.7        28.65       28.65
 28.7        28.65       28.65       28.7        28.71666667 28.76666667
 28.78333333 28.76666667 28.78333333 28.76666667 28.71666667 28.7
 28.65       28.65       28.7        28.65       28.65       28.7
 28.65       28.65       28.63333333 28.53333333 28.43333333 28.41666667
 28.4        28.45       28.56666667 28.66666667 28.81666667 28.93333333
 29.         28.93333333 28.81666667 28.73333333 28.68333333 28.66666667
 28.61666667 28.55       28.55       28.61666667 28.66666667 28.68333333
 28.73333333 

In [4]:
# 音乐库位置
audioList = os.listdir('music_base')

raw_audioList = {}
beat_database = {}

for tmp in audioList:
    audioName = os.path.join('music_base', tmp)
    if audioName.endswith('.wav'):
        y, sr = librosa.load(audioName)
        # 跟踪歌曲的节奏点
        tempo, beat_frames = librosa.beat.beat_track(y=y, sr=sr)
        # 对提取的节奏序列进行差分
        beat_frames = librosa.feature.delta(beat_frames)
        beat_database[audioName] = beat_frames

# 保存音乐节奏数据库
np.save('beatDatabase.npy', beat_database)

In [6]:
audioList = os.listdir('music_base')
print(audioList)

['彩虹.wav', '我不配.wav', '蒲公英的约定.wav', '阳光宅男.wav', '青花瓷.wav']


In [5]:
print(beat_database)

{'music_base\\彩虹.wav': array([17.55      , 17.55      , 17.55      , 17.55      , 17.55      ,
       17.53333333, 17.51666667, 17.5       , 17.41666667, 17.41666667,
       17.41666667, 17.41666667, 17.56666667, 17.56666667, 17.5       ,
       17.51666667, 17.46666667, 17.35      , 17.31666667, 17.36666667,
       17.35      , 17.43333333, 17.4       , 17.31666667, 17.33333333,
       17.3       , 17.28333333, 17.35      , 17.43333333, 17.53333333,
       17.63333333, 17.65      , 17.58333333, 17.51666667, 17.45      ,
       17.38333333, 17.4       , 17.41666667, 17.5       , 17.58333333,
       17.6       , 17.55      , 17.5       , 17.51666667, 17.45      ,
       17.38333333, 17.4       , 17.43333333, 17.35      , 17.36666667,
       17.38333333, 17.46666667, 17.48333333, 17.58333333, 17.6       ,
       17.68333333, 17.75      , 17.63333333, 17.56666667, 17.5       ,
       17.45      , 17.33333333, 17.36666667, 17.33333333, 17.31666667,
       17.33333333, 17.31666667, 17.4    

In [5]:
# -*- coding: utf-8 -*-
from dtw import dtw
from numpy.linalg import norm
from numpy import array
import numpy as np
import librosa
import pyaudio
import wave

In [9]:
# 读入音乐节奏数据库
all_data = np.load('beatDatabase.npy', allow_pickle=True)
beat_database = all_data.item()

# 读入要识别的歌曲
testAudio = "test.wav"
y, sr = librosa.load(testAudio)

# 识别歌曲的节奏序列
tempo, beat_frames = librosa.beat.beat_track(y=y, sr=sr)
beat_frames = librosa.feature.delta(beat_frames)
x = array(beat_frames).reshape(-1, 1)

# 将待识别的歌曲序列与音乐库中歌曲逐一做DTW对比
compare_result = {}
for songID in beat_database.keys():
    y = beat_database[songID]
    y = array(y).reshape(-1, 1)
    dist, cost, acc, path = dtw(x, y, dist=lambda x, y: norm(x - y, ord=1))
    print('两首歌的差异程度为： ', songID.split("\\")[1], ": ", dist)
    compare_result[songID] = dist

matched_song = min(compare_result, key=compare_result.get)
print("最接近的歌曲是：", matched_song)

两首歌的差异程度为：  彩虹.wav :  4064.5833333333358
两首歌的差异程度为：  我不配.wav :  0.0
两首歌的差异程度为：  蒲公英的约定.wav :  6971.183333333329
两首歌的差异程度为：  阳光宅男.wav :  2325.3666666666677
两首歌的差异程度为：  青花瓷.wav :  17.06666666666674
最接近的歌曲是： music_base\我不配.wav


In [ ]:
sr = 44100
chunk = sr
p = pyaudio.PyAudio()
stream = p.open(format=pyaudio.paInt16,
                channels=1,
                rate=sr,
                input=True,
                frames_per_buffer=chunk)
frames = []
for i in range(0, int(sr / chunk * 30)):
    data = stream.read(chunk)
    frames.append(data)
    
stream.stop_stream()
stream.close()
p.terminate()

wf = wave.open('test.wav', 'wb')
wf.setnchannels(1)
wf.setsampwidth(p.get_sample_size(pyaudio.paInt16))
wf.setframerate(sr)
wf.writeframes(b''.join(frames))
wf.close()